# Word game
https://wiki.apterous.org/Letters_game

Contestant chooses 9 letters by selecting either a vowel or a consonant until there is a total of 9 letters. The player can choose the letters in any order, but the selection must include at least 4 consonants and 3 vowels, hence there are only three valid choices in modern Countdown: 3 vowels, 6 consonants; 4 vowels, 5 consonants; and 5 vowels, 4 consonants

Distribution of letters: http://www.thecountdownpage.com/letters.htm

Steps:
1. Create the letter distribution
2. Select the letters. There must be a minimum of 3 vowels and minimum of 4 consonants. Maximum of 5 vowels and 6 consonants. 
3. Create the checker. One to check the user input. Another to find if there's a longer word. 

# Letter Pickers

In [1]:
#https://stackoverflow.com/questions/2570690/python-algorithm-to-randomly-select-a-key-based-on-proportionality-weight
import random

def weighted_pick(dic):
    r = random.uniform(0, sum(dic.values()))
    s = 0.0
    for k, w in dic.items():
        s += w
        if r < s: return k
    return k

In [2]:
def letter_classifier(st):
    vowel_count = 0
    consonant_count = 0
    for i in st:
        if i in 'aeiou':
            vowel_count += 1
        elif i in 'bcdfghjklmnpqrstvwxyz':
            consonant_count += 1
    return vowel_count,consonant_count

In [3]:
def letter_adder(dic):
    letter = weighted_pick(dic)
    dic[letter] -=1
    return letter

# Word input

In [4]:
#word = input("What is your word?")

# Word List

In [11]:
import pickle
import os
def resource_path(relative_path):
    """ Get absolute path to resource, works for dev and for PyInstaller """
    try:
        # PyInstaller creates a temp folder and stores path in _MEIPASS
        base_path = sys._MEIPASS
    except Exception:
        base_path = os.path.abspath(".")

    return os.path.join(base_path, relative_path)
word_key = pickle.load(open(resource_path("words.pkl"), 'rb'))

In [7]:
'''import pandas as pd#make sure that pandas is uninstalled before compiling
import pickle
df = pd.read_excel("Words.xlsx")
all_words = list(df["Word(s)"].dropna())
for word in all_words:
    if "/" in word:
        all_words.remove(word)
        new_words = word.split("/")
        all_words.extend(new_words)'''

In [8]:
#pickle.dump(all_words,open("words.pkl","wb"))

# Word Checker

In [7]:
def word_check(st):
    #convert string to set in order to match alphagram
    global word_key
    st = st.upper()
    ref = word_key
    if st in ref:
        return True
    else:
        return False

In [8]:
def letter_check(st,letter_list):
    #need to check that the user input string uses only letters from the letter list and doesn't go over the limit
    #can't use sets because repeated letters are OK
    if len(st) > 9:
        return False
    word_as_list = list(st)
    letters_as_list = list(letter_list) #had to do this step or else the letter list got overwritten
    while (len(word_as_list))>0:
        if word_as_list[0] in letters_as_list:
            letters_as_list.remove(word_as_list[0])
            word_as_list.remove(word_as_list[0])
        else:
            return False
            break
    if len(word_as_list)==0:
        return True
    else:
        return False

In [9]:
def nonvalid_word(st,letter_list):
    if len(st) > 9:
        response = "Too many letters in "+ st
        return response
    word_as_list = list(st)
    temp_letters = list(letter_list) #had to do this step or else the letter list got overwritten
    while (len(word_as_list))>0:
        if word_as_list[0] in temp_letters:
            temp_letters.remove(word_as_list[0])
            word_as_list.remove(word_as_list[0])
        else:
            response = word_as_list[0].upper() +" is not in the list of acceptable letters"
            return response
            break
    if len(word_as_list)!=0:
        response = (st + "does not use the correct letters")
        return response

# Find the best word

In [23]:
def best_word(letter_list):
    matches = []
    global word_key
    for w in word_key:
        word_alpha = sorted(list(w))
        alpha = sorted([x.upper() for x in letter_list])
        while len(word_alpha) > 0:
            if word_alpha[0] in alpha:
                alpha.remove(word_alpha[0])
                word_alpha.remove(word_alpha[0])
            else:
                break
        if len(word_alpha) == 0:
            matches.append(w)
        else:
            continue
    sorted_matches = sorted(matches,key=len)
    best = sorted_matches[-1]
    return sorted_matches,best

In [13]:
from config import *
import requests
import json

In [14]:
def oxford_lookup(word):
    language = 'en-gb'
    word_id = word.lower()
    url = 'https://od-api.oxforddictionaries.com/api/v2/entries/'  + language + '/'  + word+"?fields=definitions&strictMatch=false"
    r = requests.get(url, headers = {'app_id' : app_id, 'app_key' : app_key})
    defs = []
    if r.status_code == 200:
        for i in r.json()["results"][0]["lexicalEntries"][0]["entries"][0]["senses"]:
            defs.append(i["definitions"][0].capitalize())
    else:
        defs.append("Definition not found. I don't want to pay for more features. Sorry!")
    return defs

# all together

In [17]:
# next step: into tkinter
#start button
#buttons for vowel and consonant
#30 second timer
#box for typing in answer
#print out the response

In [18]:
from tkinter import *
from tkinter import ttk

In [19]:
def pick_vowels():
    global vowels
    global x
    global directions
    if len(x.get())<9:
        num_vowels = letter_classifier(x.get().lower())[0]
        if num_vowels < 5:
            let = letter_adder(vowels).upper()
            x.set(x.get()+let)
        else:
            directions.set(directions.get()+"\nMax vowels already reached!")
    else:
        directions.set(directions.get()+"\nMax letters reached!")
        

def pick_consonants():
    global consonants
    global x
    global directions
    num_consonants = letter_classifier(x.get().lower())[1]
    if len(x.get())<9:
        if num_consonants < 6:
            let = letter_adder(consonants).upper()
            x.set(x.get()+let)
        else:
            directions.set(directions.get()+"\nMax consonants already reached!")
    else:
        directions.set(directions.get()+"\nMax letters reached!")

In [28]:
def eval_word():
    global x
    global word
    global result
    letters = list(x.get().lower())
    if letter_check(word.get().lower(),letters) and word_check(word.get().lower()):
        result.set(result.get()+"\n"+word.get().upper()+" ("+str(len(word.get()))+") "+" is acceptable")
    else:
        result.set(result.get()+"Word is not valid. "+str(nonvalid_word(word.get().lower(),letters)))
    possible, bestword = best_word(letters)
    possible_str = ", ".join(possible)
    result.set(result.get()+"\nPossible words were: "+possible_str+"\nA good word: "+ bestword +" ("+str(len(bestword))+")")
    definition = oxford_lookup(bestword)
    if definition is None:
        result.set(result.get()+"\nDefinition not found")
    else:
        word_def = str(definition)
        result.set(result.get()+"\n"+word_def)
        


In [24]:
def reset():
    global default_directions
    global default_x
    global default_word
    global default_result
    global directions
    global x
    global word
    global result
    directions.set(default_directions)
    x.set(default_x)
    word.set(default_word)
    result.set(default_result)

In [29]:
root = Tk()
root.title("Letters Game")

mainframe = ttk.Frame(root, padding="3 3 12 12")
mainframe.grid(column=0, row=0, sticky=(N, W, E, S))
root.columnconfigure(0, weight=1)
root.rowconfigure(0, weight=1)

vowels = {"a":15,"e":21,"i":13,"o":13,"u":5}
consonants = {"b":2,"c":3,"d":6,"f":2,"g":3,"h":2,"j":1,"k":1,"l":5,"m":4,"n":8,"p":4,"q":1,"r":9,"s":9,"t":9,"v":1,"w":1,"x":1,"y":1,"z":1}

#default strings
default_directions = "Welcome to the Letter Game!"+"\nChoose 9 letters using the vowel and consonant buttons and form the longest word possible."+" Evaluate the word. See how close you get!"+"\nTODO: add the 30 second time limit. You can time yourself though"
default_x = ""


directions = StringVar(value=default_directions)
x = StringVar()
    
ttk.Label(mainframe, text=directions.get(), textvariable = directions,wraplength=500).grid(row=0, column=0,columnspan=6)
ttk.Label(mainframe, text=x.get(), textvariable = x).grid(row=1, column=1,columnspan=2)
ttk.Button(mainframe, text='Vowel', command=pick_vowels).grid(row=2, column=1)
ttk.Button(mainframe, text='Consonant', command=pick_consonants).grid(row=2, column=2)

ttk.Label(mainframe,text="Enter your word here:").grid(row=3,column=0)

default_word = ""
word = StringVar()
word_entry = ttk.Entry(mainframe, textvariable=word)
word_entry.grid(row=3,column=1,columnspan=2)

default_result = ""

ttk.Button(mainframe,text="Evaluate!",command=eval_word).grid(row=3,column=3)

ttk.Button(mainframe,text="Reset",command=reset).grid(row=4,column=3)

result = StringVar()
ttk.Label(mainframe, text=result.get(),textvariable=result,wraplength=500).grid(row=5,column=0,columnspan=6)

for child in mainframe.winfo_children(): 
    child.grid_configure(padx=5, pady=5)
    
root.mainloop()

Exception in Tkinter callback
Traceback (most recent call last):
  File "/Users/mathuser/anaconda3/envs/wordsnumbers/lib/python3.9/tkinter/__init__.py", line 1892, in __call__
    return self.func(*args)
  File "/var/folders/nd/m4dyffh95zncvzwr4srtj0rm0000gp/T/ipykernel_10679/295111930.py", line 13, in eval_word
    definition = oxford_lookup(bestword)
  File "/var/folders/nd/m4dyffh95zncvzwr4srtj0rm0000gp/T/ipykernel_10679/4238594102.py", line 8, in oxford_lookup
    for i in r.json()["results"][0]["lexicalEntries"][0]["entries"][0]["senses"]:
KeyError: 'entries'
